In [1]:
import os
import torch
from torch.utils.data import Dataset
import torchaudio
import soundfile
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment

c:\Users\hause\miniconda3\envs\practical_work\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [36]:
class ChunkedMusDBHQ(Dataset):

    def __init__(self, audio_dir) -> None:
        self.audio_dir = audio_dir

    def __len__(self):
        count = 0
        # Iterate directory
        for path in os.listdir(self.audio_dir):
            # check if current path is a file
            if os.path.isfile(os.path.join(self.audio_dir, path)):
                count += 1
        return count

    def __getitem__(self, index):
        file = os.listdir(self.audio_dir)[index]
        waveform, sample_rate = torchaudio.load(self.audio_dir + file)
        return waveform, sample_rate

In [60]:
chunked_data = ChunkedMusDBHQ(audio_dir='data/chunked_audio/')

trainloader = torch.utils.data.DataLoader(
    chunked_data,
    batch_size=2,
    shuffle=True,
    )

In [2]:
musdb_train = torchaudio.datasets.MUSDB_HQ(
    root="data", 
    download=True,
    subset='train',
    sources=['mixture'])

In [3]:
trainloader = torch.utils.data.DataLoader(
    musdb_train,
    batch_size=1,
    shuffle=True,
    )

In [63]:
sample = next(iter(trainloader))

In [64]:
waveform = sample[0]
sample_rate = sample[1]
track_name = sample[3]
track_name

('Music Delta - Rockabilly',)

In [65]:
sample[0][0].shape

torch.Size([1, 2, 1144358])

In [26]:
#load data of one song and split it into 5 second chunks
path = os.path.join('data','musdb18hq','train','A Classic Education - NightOwl','mixture.wav')
audio = AudioSegment.from_wav(path)

n_iters = int(len(audio) / 5000)

start = 0
end = 5000

for i in range(n_iters):
    newAudio = audio[start:end]
    newAudio.export('A Classic Education - NightOwl_{0}.wav'.format(start), format="wav")
    start += 5000
    end += 5000
